# **LTI&trade; Advantage** bootcamp notebook for Tool

## Introduction

The notebook shows how to interact with the LTI Advantage ecosystem from a tool implementer viewpoint. It interacts with an actual test server which has been built as a platform simulator to support this notebook.


## Setup

Here we just import the libraries that will be needed in this notebook, define some utility functions and constants.

In [111]:
import requests
import json
import jwt
import base64
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicNumbers
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from IPython.display import display, Markdown

def decode_int(b64value):
    return int.from_bytes(base64.urlsafe_b64decode(b64value), byteorder='big')

# for concise code, return full claim prefixed by ims
def fc(claim):
    return "http://imsglobal.org/lti/{0}".format(claim)

def md(mdt):
    display(Markdown(mdt))
    
md_buffer = ''

def mdb(mdt=None):
    global md_buffer
    if mdt:
        md_buffer = md_buffer + '\n' + mdt
    else:
        md(md_buffer)
        md_buffer = ''

# This notebook queries an actual test server. It needs its location:
platform_url='http://localhost:5000'


First we need to get a new tool deployment from the server for this notebook instance to use.
Each tool must have a client_id and a private key that will be used to interact with the platform services and send messages back to the platform.
It also needs the keyset URL that exposes the platform public keys needed to validate the incoming messages.

While this information is required for each tool, how it is obtained by the tool is NOT currently part of the LTI specifications.

In [143]:
r = requests.get(platform_url + '/newtool')
tool_info = json.loads(r.text)

md('### Tool information')
md('Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.')
md('```json\n'+ json.dumps(tool_info, indent=4)+'```')

### Tool information

Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.

```json
{
    "client_id": "0",
    "deployment_id": "deployment_0",
    "key": {
        "e": "AQAB",
        "kty": "RSA",
        "use": "sig",
        "alg": "RS256",
        "d": "X5O4Ad4fEO2OiGIRplQlHywX_rqKukf2HzlM1cl9i5PR7elPbj_JwLE3ilDEY52oJgFoXL7eKiXMi3lV7bQpEz8pD2TUN-ZfTksX250uTH7T3pdffyYH3S4B9NTl6gbn7R5X3madqetvy0oZ8GR9Si_KyXTznrnWJNo6LJCKs4-dPNt4n2TFUSTPi8JoOWk66Q5L0T5IwFHtBHbXgq5b7T2OkrWP7T-N0CjGO1HcZuKHGxMtJmZdAXK586X1PiY_Pwy37YrHatTXGwmL-AK4SpZzWcM-vBlR5-E1Zhrjmr1xw04HRJtpaolPe2H13ACIn-zyb44o61z0MHF5uZyyEQ=="
    }
}```

### Getting the public keys from the platform

In order to validate the various messages we will receive from the platform, we need to get the key sets. We'll also transform the keys to the `PEM` format that is used by the `jwt` module to decode the messages.

The test platform exposes its keys in a keyset format at a well-know location (.well-known/jwks.json). Other platform might just communicate the keyset url as part of the tool deployment information.

In [144]:
keyset = json.loads(requests.get(platform_url + '/.well-known/jwks.json').text)

md('#### Platform keyset')
md('```json\n'+ json.dumps(keyset, indent=4)+'```')

platform_keys = {}

# let's transform as a map for ease of use, and just the PEM because this is what is used by JWT lib
for key in keyset['keys']:
    public_key = RSAPublicNumbers(decode_int(key['e']), decode_int(key['n'])).public_key(default_backend())
    pem = public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)
    platform_keys[key['kid']] = pem
    


#### Platform keyset

```json
{
    "keys": [
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "kgaCSJ0X4TnruUjHDl-bADcYCRehKgf7pDsJeSWVpdxuaoIzwf8W2vpUqTD0i1xSrZ0dcGLFO9IMAA08ipbaB7XnIgevbc_PAXsfjK6xDcRS7tNtATtcjs-80SAceXslz92fg3M-6Ur3FhfgVjNigP7LCTT9FcJO7OZvRO4YRZcWTH2-whxfK45JfSuiYQj3PxSznX4hQlDG5PiwZBBXR5_m_AsX1gbhAj186yfExjgOmM7Wk4z0-5RyNt9dE-Z2yelqcVoNY42kiwtE7zB3R81S-jYnfM_zTzU6GYbCCgZLmxE25MmphjowTXqyzFRfqG806hfpSnBh3-XH3OR8UQ==",
            "kid": "1517533449_0"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "sYCeSp5Cs1Tr5n0w2eH1PsfGwzr8YZskvSudpcDM8abi05c2W7xr2YW7GETfKAb7ypvDwTn4kK1yq1bk-0dy074GqIWZ3WGAidRJpIYmkV0IZKYWkmGmRg2GwYxbepHhNaTfQjDNfvWNhfbaQg_fIvV0ZHdrYKrTRsAhbGpB_xp1D9hIvC9yVLLHWDhket3YkbM_h0V8czX0Q4ooxTw4w3zVGeVQmH0UW4w5C5WgoFPM1o5D3jEUOcIkrvZu43ZATxNTJP0vl7GIaG_hvE66E5PD1Q1jKQ5we_3I3qgMdGaFeNU95zAcrdKJB_wnQglLKcFtx76Lr6laPnIuzT_MbQ==",
            "kid": "1517533449_1"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "ohmRx-cM0x8eBUGd8cV2GTy0Xc-7Rlo9EmV53Uwf9QhwvHAy8qWyEuhXjmdQSucsHHLMLPz98Z82DA2ZEIzqnajwQoQ11dgUX_LxvZ8KQVn8A4ICFMYU7jwG_wohS9o7yay6uzgNtPoVVCpoU5w4kwBm8Mq8-uiuygEqZS_ob4cMQ3iy0udFqFGW9eg4B-ZvfU26ASRYecZwUmgVS40QPOaCFo1RtgGELcgcYHR6IarUFA3VAeglA4p8Fp33pxfRvZ14oSKDeMf5RqKZNcznm8Jp90Gt_WER8AY0RqZ_hf3K3pNfGz5nQS8T-77nO3Z0BTX4egKW0xKENLlaTs3-YQ==",
            "kid": "1517533449_2"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "rLR_sy2KOL443VaTpl6G0bA8RUhB5fbvMq8_5YGDlgLU10ufNk1vVVTWPAnefOLM-pngqY6J6hw1IfKZjXPRDs-b6qp7VovSqNZHmDGE8cL_-KcyunmnMLYqgLfvfRZ3ZuNcOqoWXw3C-WpW4c1U3XzkCn67ARmqlSjg4f0B8UtkPtIDbvJ8mIKdYKg_rVVDPGBo-A-6qZrU-_LUFigsfZuvCvf_G_fSUtiuNESgBJOezQV0ktxBL60_fQyxrki_v39Z1t7kYhY03Pbn2hAzqoxzLF_bT-k-tAIhbSTNWOypOdtHzZzRIkUpU6tQhrzUAxXDkUd9dtBSXHq_Te3QxQ==",
            "kid": "1517533449_3"
        }
    ]
}```

## Deep Linking - Creating a Link
This section will use the deep linking specification to create a Resource Link to the platform. That resource link will be gradable and used in the following sections of that notebook.

[Deep Linking](https://www.imsglobal.org/specs/lticiv1p0) is a **UI flow**, it is an important piece that is sometimes missed on 1st reading. The user is redirected from the platform to the tool to pick or create one or multiple piece of content (often, LTI links), and the the tool redirects the UI back to the tool with the actual selection (or an empty payload if nothing was picked or created).

So there are 2 messages:

1. `ContentItemRequestSelection` from the platform to the tool to start the picking/create session. This is a typical platform launch that contains the context and the user information, and what kind of content items may be created in this flow (for example, this flow might indicate it only wants LTI links).
1. `ContentItemSelection` from the tool back to the plaform using the `content_item_return_url` provided in the request.

Once a tool is added to a course, usually the first launch from the platform will be a Deep Linking request.

![deep linking](assets/lti_advantage_deeplinking.png "LTI Deep Link")

### Setup: Getting a Deep Linking Request

This notebook is not a tool actually launched by the platform, so the test platform as way to give us the token that it would include in an actual HTTP POSTrequest, so we can build a mock POST request including the parameter `post_data`.

In [145]:
r = requests.get("{}/tool/{}/cisr".format(platform_url, tool_info['client_id']))

post_data = {
    'id_token':r.text
}

### Task 1: Verify the JWT is properly signed

The first thing before to display to the user the picker/authoring interface to create the link is to validate this request is properly signed. This is done by decoding the JWT using public key from the platform.


In [146]:
# Let's get the kid so we can get the proper public key
encoded_jwt = post_data['id_token']
jwt_headers = jwt.get_unverified_header(encoded_jwt)

content_item_message = jwt.decode(encoded_jwt, 
                                  platform_keys[jwt_headers['kid']], 
                                  jwt_headers['alg'],
                                  audience = tool_info['client_id'])

md('#### Message properly signed! Decoded ContentItemSelectionRequest message:')
md('```json\n'+ json.dumps(content_item_message, indent=4)+'```')


#### Message properly signed! Decoded ContentItemSelectionRequest message:

```json
{
    "http://imsglobal.org/lti/version": "1.3.0",
    "iss": "http://localhost:5000",
    "sub": "LTIBCU_12",
    "http://imsglobal.org/lti/context": {
        "title": "LTI Bootcamp Course",
        "type": [
            "CourseSection"
        ],
        "id": "ffe7b882-07b4-11e8-a22c-c48e8ffb7857",
        "label": "LTI Bootcamp Course"
    },
    "http://imsglobal.org/lti/roles": [
        "http://purl.imsglobal.org/vocab/lis/v2/membership#Instructor"
    ],
    "aud": "0",
    "family_name": "Talulla",
    "iat": 1517533470,
    "http://imsglobal.org/lti/deep_linking_request": {
        "accept_media_types": [
            "application/vnd.ims.lti.v1.ltilink"
        ],
        "accept_multiple": true,
        "auto_create": true,
        "data": "op=321&v=44",
        "accept_presentation_document_targets": [
            "iframe",
            "window"
        ]
    },
    "http://imsglobal.org/lti/message_type": "ContentItemSelectionRequest",
    "nonce": "05d66bb2-07b5-11e8-a22c-c48e8ffb7857",
    "http://imsglobal.org/lti/deployment_id": "deployment_0",
    "email": "Linwood.Talulla@example.com",
    "exp": 1517533530,
    "http://imsglobal.org/lti/tool_platform": {
        "guid": "ltibc_at_1517533450",
        "name": "LTI Bootcamp Platform"
    },
    "given_name": "Linwood",
    "name": "Linwood Talulla"
}```

### Task 2: extract the information to decide which experience to show

If this is the first launch for the user or the course, as a tool you may prompt the user for setup information, including account linking or course setup. Ultimately the user will see the authoring or picking interface that will allow her to create or select the content items to be added to the course.

Some key attributes of the `ContentItemSelectionRequest` will drive the experience:

In [150]:

mdb('1. What is the current course id? {0}'.format(content_item_message[fc('context')]['id']))
mdb('1. What is the current user id? {0}'.format(content_item_message['sub']))
is_instructor = len(list(filter(lambda x: 'instructor' in x.lower(), content_item_message[fc('roles')])))>0
mdb('1. Is this user an instructor? {0}'.format(is_instructor))
deep_linking_claim = content_item_message[fc('deep_linking_request')]
mdb('1. What kind of content item can be created? {0}'.format(deep_linking_claim['accept_media_types']))
mdb('1. Can I return more than one items to be added? {0}'.format(deep_linking_claim['accept_multiple']))
mdb('1. Will the user be prompted before to actually save the items? {0}'.format(not deep_linking_claim['auto_create']))
mdb()
md('1. Where should I redirect the browser too when done? {0}'.format(content_item[c('')]))
md('1. Is there any data I must pass back to platform when I return? {0}'.format(content_item[c('')]))


1. What is the current course id? ffe7b882-07b4-11e8-a22c-c48e8ffb7857
1. What is the current user id? LTIBCU_12
1. Is this user an instructor? True
1. What kind of content item can be created? ['application/vnd.ims.lti.v1.ltilink']
1. Can I return more than one items to be added? True
1. Will the user be prompted before to actually save the items? False

NameError: name 'content_item' is not defined

### Task 3: redirect the user back to the platform with the content item selection

After the end of the interaction, so user is sent back to the platform throught a browser redirection using an HTTP POST containing the JWT `ContentItemResponse` message. In this case, we will return 2 LTI links, one being graded.

In [ ]:
# Let's start by adding the JWS security claims
content_item_response = {
    'iss': tool_info.client_id,
    'aud': content_item_message['iss']
}